# INaturalist component.

- Should we be using iNaturalist to get the public more engaged? 
  + to few entries at current for use in identifying Burmese Python patterns.[82 total in FL]
  + could be usable, though almost half the users are anonimous. [28 of the 55 totalusers have a name attached to account]
- Which iNaturalist users are most active in sighting pythons? (edited) 
  + mostly nul on actual names. top3
      - pythonbob	   - noname give  : 12
      - joemdo	        - Joe MDO     : 6
      - michiko         - Michiko     : 4
- Which iNaturalist users are most active in identifying pythons? Hint: iNaturalist observations must be confirmed by other users... (edited) 
      - daniel_e    - Daniel Estabrooks :65
      - tysmith	    - no name given :42
      - gibbous	    - no name given :38     
- Which iNaturalist users are most connected in the python-spotting community (i.e. who are the influencers)? (edited) 
      - gibbous	   - no name give       
      - dodomae98  - no name given
      - pythonbob  - no name given
- Are any wildlife officials also using iNaturalist? (Hint: yes)
     - 3 peole. 

In [13]:
import re
import networkx as nx
import pandas as pd
import networkx as nx
import pandas as pd
import numpy as np
import requests 
from pandas.io.json import json_normalize
from math import ceil
import json
# from json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
def top5_dict(in_put, top = 5):
    in_sort = sorted(in_put.items(), key=lambda kv: kv[1],reverse = True)
    top_ones =[el for i,el in enumerate(in_sort) if i<top]
    return top_ones

In [3]:
dataurl = "https://api.inaturalist.org/v1/observations?verifiable=true&order_by=observations.id&order=desc&spam=false&place_id=21&taxon_id=238252&locale=en-US&per_page=100"
responce = requests.get(dataurl)
responce

<Response [200]>

In [4]:
data = responce.json()
flatdata = json_normalize(data)
# flatdata
total=100
per_page = flatdata['per_page']
pages = ceil(total/ per_page)
# pages
page_data = []
for page in range(1,pages+1):
    responce = requests.get(dataurl + '&page='+ str(page))
    result = json_normalize(responce.json()['results'])
    page_data.append(pd.DataFrame(result))
        
df = pd.concat (page_data)
identifications= df['identifications']
# other fix to nest: cosider the use of postman to 
df.drop(columns= ['identifications','non_owner_ids'],inplace = True)
col=[str(i).replace('.','_') for i in list(df.columns)]
df.columns=col # replaces '.' with '_' to facilitate queries.

### Post Count
- code bellow determines the username of the top 5 observation posters for burmese pythons in FL on INaturalist.

In [10]:
df.groupby(['user_login_exact']).count().sort_values('annotations',ascending=False)['identifications_count'].head(5)

user_login_exact
pythonbob              12
joemdo                 6 
atrox77                4 
michiko                4 
snakesareprettycool    3 
Name: identifications_count, dtype: int64

##### The top 5 most identifications posted

In [9]:
really = pd.DataFrame(relation, columns = ['post_ID','identifier_username'])
really.groupby(['identifier_username']).count().sort_values('post_ID',ascending=False).head(5)

,post_ID
identifier_username,
daniel_e,65
tysmith,42
gibbous,38
bobbyfingers,32
dodomae98,29


### relationship. 
- code bellow grabs the identifications made by poeple for each entrie and calculates who is connected to the most people by the least number of steps. i other words who can be considered the most influential in the python comunity of INaturalist 

In [5]:
relation = []
for i in range(82):
    idet  = pd.DataFrame(identifications.values.flatten()[i])
    for user in idet['user']:
        rel = re.sub("', 'spam'.*",'',str(user))
        rel = re.sub(".*'login': '",'',rel)
        relation.append([i,rel]) # cleaning data for use
rel = []
for i in range(82):
    rel.append(relation[i])# = df['user.login_exact'].loc[i] # list of names by id they identified for
jofson = pd.DataFrame(rel, columns = ['Name_x', 'Name_y'])

In [6]:
G = nx.convert_matrix.from_pandas_edgelist(jofson, source = 'Name_x', target = 'Name_y')

top5_dict( dict(nx.degree_centrality(G)))

[('gibbous', 0.36170212765957444),
 ('dodomae98', 0.2553191489361702),
 ('lucareptile', 0.1276595744680851),
 (9, 0.1276595744680851),
 (13, 0.1276595744680851)]

In [7]:
top5_dict(dict(nx.eigenvector_centrality_numpy(G)))

[('gibbous', 0.48472995659963836),
 ('dodomae98', 0.383228512267044),
 (14, 0.21729486757533506),
 (13, 0.20095779569687705),
 (10, 0.20059193985240603)]

In [8]:
top5_dict(dict(nx.betweenness_centrality(G,weight = 'edge')))

[('gibbous', 0.5595063336234998),
 ('dodomae98', 0.17609914155172438),
 (5, 0.12616661527392334),
 (4, 0.11366532774767954),
 (3, 0.10656496796949733)]